In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

init_vprinting()

### Robotics Toolbox Model

In [ ]:
from roboticstoolbox.models.DH import Panda, Puma560

In [ ]:
panda = Panda()

In [ ]:
panda

In [ ]:
def tau_from_state(q, qd, qdd):
    panda = Panda()
    M = panda.inertia(q)
    C = panda.coriolis(q, qd)
    G = panda.gravload(q)
    tau = M@qdd + C@qd + G
    return tau

In [ ]:
def randq():
    panda = Panda()
    qlim = np.zeros([7,2])
    for ii, link in enumerate(panda.links):
        qlim[ii] = link.qlim
    qlim
    r = np.random.random_sample(7)
    d = qlim[:,1]-qlim[:,0]
    c = qlim[:,0]
    return c + d*r
    

In [ ]:
def randpath(N):
    path = np.zeros([N,7])
    for ii in range(N):
        path[ii,:] = randq()
    return np.round(path, 3)

In [ ]:
def rand_traj(N):
    traj = np.zeros([N, 15])
    traj[:,:7] = randpath(N)
    traj[1:-1, 7:14] = np.round((traj[2:,:7] - traj[:-2,:7])/2, 3)
    traj[:,-1] = np.arange(N)
    return traj

In [ ]:
from optibot.schemes import extend_array

In [ ]:
def savetraj(N, fname = 'ejtraj.txt'):
    traj = extend_array(rand_traj(N))
    traj[-1, -1] += 0.2
    np.savetxt(fname, traj, '%6.3f')
    return traj

def loadtraj(fname = 'ejtraj.txt'):
    return np.loadtxt(fname)

### Generating, saving and loading a random trajectory

In [ ]:
savetraj(7)
ej_traj = loadtraj()

In [ ]:
from scipy.interpolate import CubicHermiteSpline as hermite

In [ ]:
her_traj = hermite(ej_traj[:,-1], ej_traj[:, :7], ej_traj[:, 7:14])
her_speed = her_traj.derivative()
her_accel = her_speed.derivative()
interp_f = [her_traj, her_speed, her_accel]

In [ ]:
plt.style.use('default')
t_plot = np.linspace(0, ej_traj[-1, -1], 500)
for ii, varname in enumerate(['q', 'q_dot']):
    plt.figure(figsize=[12,8])
    plt.title('ideal ' + varname)
    plt.plot(ej_traj[:,-1], ej_traj[:, 7*ii: 7*(ii+1)], 'o')
    plt.plot(t_plot, interp_f[ii](t_plot))
    plt.grid()

In [ ]:
interp_n = 500
t_plot = np.linspace(0, ej_traj[-1, -1], interp_n)
tau_arr = np.zeros([interp_n, 7])
for ii in range(interp_n):
    _t = t_plot[ii]
    _q = her_traj(_t)
    _qd = her_speed(_t)
    _qdd = her_accel(_t)
    _tau = tau_from_state(_q, _qd, _qdd)
    tau_arr[ii,:] = _tau

plt.figure(figsize=[12,8])
plt.title('ideal torque')
plt.plot(t_plot, tau_arr)
plt.grid()